### Dataset Choice
For this exercise, I selected the **Job Posting Data in Honduras** dataset from Kaggle.  
It provides job posting information from multiple sources and includes fields like job title, company, location, category, and posting dates.  

[Kaggle Dataset Link](https://www.kaggle.com/datasets/techsalerator/job-posting-data-in-honduras)

In [52]:
import pandas as pd
import numpy as np

In [53]:
df = pd.read_csv("../data/Job Posting.csv", encoding="latin1")

print(df.head(10))

  Website Domain  Ticker                                  Job Opening Title  \
0      bosch.com     NaN  IN_RBAI_Assistant Manager_Dispensing Process E...   
1      bosch.com     NaN  Professional Internship: Hardware Development ...   
2         zf.com     NaN                      Process Expert BMS Production   
3      bosch.com     NaN  DevOps Developer with Python for ADAS Computin...   
4      bosch.com     NaN  Senior Engineer Sales - Video Systems and Solu...   
5      bosch.com     NaN  [EM] _______ _________ (________ EV/HEV Compon...   
6      bosch.com     NaN                                Controls Technician   
7    heraeus.com     NaN        Intern (m/f/d) Digital Business & eCommerce   
8      bosch.com     NaN  ANALISTA DESENVOLVIMENTO SOFTWARE SR (ServiceN...   
9      bosch.com     NaN           Production Technician - Bosch Charleston   

                                     Job Opening URL         First Seen At  \
0  https://jobs.smartrecruiters.com/BoschGroup/74...

In [54]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9919 entries, 0 to 9918
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Website Domain         9919 non-null   object 
 1   Ticker                 0 non-null      float64
 2   Job Opening Title      9919 non-null   object 
 3   Job Opening URL        9919 non-null   object 
 4   First Seen At          9919 non-null   object 
 5   Last Seen At           9919 non-null   object 
 6   Location               9508 non-null   object 
 7   Location Data          9919 non-null   object 
 8   Category               8250 non-null   object 
 9   Seniority              9919 non-null   object 
 10  Keywords               7646 non-null   object 
 11  Description            9807 non-null   object 
 12  Salary                 576 non-null    object 
 13  Salary Data            9919 non-null   object 
 14  Contract Types         8004 non-null   object 
 15  Job 

In [55]:
print(df.isnull().sum())

Website Domain              0
Ticker                   9919
Job Opening Title           0
Job Opening URL             0
First Seen At               0
Last Seen At                0
Location                  411
Location Data               0
Category                 1669
Seniority                   0
Keywords                 2273
Description               112
Salary                   9343
Salary Data                 0
Contract Types           1915
Job Status               3147
Job Language                2
Job Last Processed At       0
O*NET Code                  3
O*NET Family                3
O*NET Occupation Name       3
dtype: int64


In [56]:
df.describe()

,Ticker
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


### Top 2–3 Problems Identified

After performing the initial data diagnosis using `df.info()`, `df.isnull().sum()`, and `df.describe()`, several critical data quality issues were identified:

1. **Completely Empty Columns**  
   - The `Ticker` column was entirely empty (100% missing). This makes it unusable for any type of analysis, so it must be removed.

2. **Columns with Extremely High Missing Rates**  
   - The `Salary` column contained over 94% missing values, combined with inconsistent formats (different currencies, hourly/monthly/yearly time bases). This made the column unreliable and not worth cleaning for this exercise.

3. **Categorical Columns with Missing Values**  
   - Several categorical columns — including `Category`, `Keywords`, `Contract Types`, and `Job Status` — had a significant number of missing values. These needed to be filled with a consistent placeholder to avoid data loss.

4. **Date Columns Stored as Strings**  
   - Date fields such as `First Seen At`, `Last Seen At`, and `Job Last Processed At` were stored as object (string) types. This prevented time-based analysis and required conversion to proper datetime format.

5. **Minor Missing Values in Key Columns**  
   - Columns like `O*NET Code`, `O*NET Family`, `O*NET Occupation Name`, and `Job Language` had only a handful of missing rows compared to the dataset size (~9,900 rows). These could be safely dropped to maintain column integrity.

---

**Summary:**  
The dataset required targeted cleaning steps:
- Dropping entirely empty or unreliable columns (`Ticker`, `Salary`)  
- Filling categorical missing values with `"Unknown"`  
- Dropping a few rows with critical missing values  
- Converting date columns to `datetime` type

After applying these transformations, the dataset became **complete, consistent, and analysis-ready**, with no missing values remaining and proper data types assigned.


#### Dropping Completely Empty or Unusable Columns
The Ticker column was entirely empty (100% missing), and the Salary column had over 94% missing values with inconsistent formats (currencies, time bases).
Both columns were dropped, since they provide no meaningful information for analysis and would complicate cleaning.

In [57]:
df.drop(columns=['Ticker'], inplace=True)
df.drop(columns=['Salary'], inplace=True)

#### Filling Missing Values in Categorical Columns
Several categorical columns — such as Location, Keywords, Description, Contract Types, Job Status, and Category — contained missing values.
Instead of dropping these rows, we filled the missing entries with the placeholder "Unknown", ensuring consistency and avoiding data loss.

In [58]:
cols_to_fill = ['Location', 'Keywords', 'Description', 'Contract Types', 'Job Status', "Category"]
df[cols_to_fill] = df[cols_to_fill].fillna('Unknown')

#### Dropping Rows with Minimal Missing Values in Key Columns
Only a few rows (5 in total) had missing values in critical columns like O*NET Code, O*NET Family, O*NET Occupation Name, and Job Language.
Given the very small proportion of missing rows compared to the total dataset, we decided to drop them entirely rather than fill them with arbitrary placeholders.

In [59]:
df.dropna(
    subset=['O*NET Code', 'O*NET Family', 'O*NET Occupation Name', 'Job Language'],
    inplace=True
)

#### Converting Date Columns to Datetime Format
Date columns were originally stored as object type strings, which is unsuitable for time-based analysis.
We converted First Seen At, Last Seen At, and Job Last Processed At to proper datetime64[ns] format using pd.to_datetime, allowing for easier filtering, sorting, and time-series operations.

In [60]:
date_cols = ['First Seen At', 'Last Seen At', 'Job Last Processed At']

for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')


In [61]:
print(df.info())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 9914 entries, 0 to 9918
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   Website Domain         9914 non-null   object             
 1   Job Opening Title      9914 non-null   object             
 2   Job Opening URL        9914 non-null   object             
 3   First Seen At          9914 non-null   datetime64[ns, UTC]
 4   Last Seen At           9914 non-null   datetime64[ns, UTC]
 5   Location               9914 non-null   object             
 6   Location Data          9914 non-null   object             
 7   Category               9914 non-null   object             
 8   Seniority              9914 non-null   object             
 9   Keywords               9914 non-null   object             
 10  Description            9914 non-null   object             
 11  Salary Data            9914 non-null   object             
 1

### Cleaning Summary & Verification

After performing the data cleaning steps, the dataset has been significantly improved:

1. **Removed useless columns**:  
   - The `Ticker` column was completely empty (100% missing), so it was dropped.  
   - The `Salary` column had over 94% missing values with inconsistent formats, making it unreliable for analysis, so it was also dropped.

2. **Handled missing values in categorical columns**:  
   - Columns such as `Location`, `Keywords`, `Description`, `Contract Types`, `Job Status`, and `Category` had a noticeable number of missing entries.  
   - These were filled with the value `"Unknown"` to preserve the dataset size without losing rows unnecessarily.

3. **Dropped a few rows with missing critical values**:  
   - Only a handful of rows (less than 5) with missing values in `O*NET Code`, `O*NET Family`, `O*NET Occupation Name`, and `Job Language` were removed to ensure data consistency.

4. **Converted date columns to proper datetime format**:  
   - The `First Seen At`, `Last Seen At`, and `Job Last Processed At` columns were originally stored as objects (strings).  
   - These were successfully converted to `datetime64[ns, UTC]`, allowing proper temporal analysis in future steps.

---

### ✅ Final Check

- Running `df.info()` now shows **no remaining null values**.  
- The dataset has **19 clean and well-structured columns**.  
- All date columns have correct data types, and categorical columns are complete.  

Overall, the dataset is now ready for further exploration and analysis.
